<a href="https://colab.research.google.com/github/dataforgoodfr/batch8_mednum/blob/master/notebooks/acces_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Liste des médiathèques


In [ ]:
# get mediatheques map

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
ls drive/'My Drive'/'Colab Notebooks'/

carte-des-bibliotheques.csv                 mediatheque.ipynb
correspondances-code-insee-code-postal.csv


In [ ]:
cd drive/'My Drive'/'Colab Notebooks'/

/content/drive/My Drive/Colab Notebooks


In [ ]:
import pandas as pd

df = pd.read_csv('carte-des-bibliotheques.csv', header=0, sep=';')
print(df.shape)
df.head()

(844, 21)


,Nom structure,Adresse,Code postal,Département,Ville,Année(s) subvention Région,Montant subvention 2000,Montant subvention 2001,Montant subvention 2002,Montant subvention 2003,Montant subvention 2004,Montant subvention 2005,Montant subvention 2006,Montant subvention 2007,Montant subvention 2008,Montant subvention 2009,Montant subvention 2010,Montant subvention 2011,Montant subvention 2012,Montant subvention 2013,wgs84
0,MÉDIATHÈQUE SAINT-JOHN PERSE,2 rue Édouard Poisson,93300,93,AUBERVILLIERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.9104747,2.385008"
1,MÉDIATHÈQUE GEORGES PÉREC,8-10 rue du Marqis de Raies,91080,91,COURCOURONNES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6293899,2.4135832"
2,MÉDIATHÈQUE LES MUSES,101 rue de la Division Leclerc,91310,91,LINAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6305113,2.2650636"
3,MÉDIATHÈQUE MUNICIPALE,2 route de la Creusée,91530,91,LE-VAL-SAINT-GERMAIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.5657217,2.0601725"
4,BIBLIOTHÈQUE LE GRENIER,5 place des Droits de l'Homme,91800,91,BOUSSY-SAINT-ANTOINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6891076,2.5297452"


In [ ]:
df = df[['Code postal', 'Ville', 'Département']]
df['Code postal'] = df['Code postal'].astype(str)
df.head()

,Code postal,Ville,Département
0,93300,AUBERVILLIERS,93
1,91080,COURCOURONNES,91
2,91310,LINAS,91
3,91530,LE-VAL-SAINT-GERMAIN,91
4,91800,BOUSSY-SAINT-ANTOINE,91


In [ ]:
# mapping code postal - code INSEE
mapping = pd.read_csv('correspondances-code-insee-code-postal.csv', header=0, sep=';')
mapping.rename(columns={'Code Postal': 'Code postal'}, inplace=True)
mapping.head()

,Code INSEE,Code postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geofla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région
0,91645,91370,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,Commune simple,121.0,999.0,15.5,"48.75044312,2.25171297214","{""type"": ""Polygon"", ""coordinates"": [[[2.238024...",16275,645,3,3,91,11
1,77133,77126,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,Commune simple,88.0,1082.0,0.2,"48.4125606521,3.05294050556","{""type"": ""Polygon"", ""coordinates"": [[[3.076046...",31428,133,20,3,77,11
2,91378,91730,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,Commune simple,150.0,313.0,0.3,"48.5272680908,2.19718165044","{""type"": ""Polygon"", ""coordinates"": [[[2.203466...",30975,378,9,1,91,11
3,77243,77400,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,Chef-lieu canton,71.0,579.0,20.2,"48.8730701858,2.70978081313","{""type"": ""Polygon"", ""coordinates"": [[[2.727542...",17000,243,14,5,77,11
4,94003,94110,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,Chef-lieu canton,70.0,232.0,19.5,"48.8058803597,2.33351024984","{""type"": ""Polygon"", ""coordinates"": [[[2.343851...",32123,3,34,3,94,11


In [ ]:
mapping.drop_duplicates(inplace=True)

In [ ]:
df.merge(mapping[['Code INSEE', 'Code postal']], how='inner', on='Code postal').drop_duplicates()  

,Code postal,Ville,Département,Code INSEE
0,93300,AUBERVILLIERS,93,93001
4,91080,COURCOURONNES,91,91182
5,91310,LINAS,91,91333
6,91310,LINAS,91,91425
7,91310,LINAS,91,91339
...,...,...,...,...
2722,77440,ARMENTIÈRES-EN-BRIE,77,77490
2723,77440,ARMENTIÈRES-EN-BRIE,77,77231
2724,77440,ARMENTIÈRES-EN-BRIE,77,77235
2725,77200,TORCY,77,77468


In [ ]:
df.groupby('Code postal').count().sort_values(by='Ville')[::-1]

,Ville,Département
Code postal,,
78000,9,9
77400,7,7
77130,7,7
78490,6,6
95000,6,6
...,...,...
92104,1,1
92110,1,1
92120,1,1


## 2. Liste des centres sociaux

In [ ]:
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib import request

import time


def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()        
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result    
    return timed

In [ ]:
@timeit
def scrap_table_page(url='https://www.senacs.fr/structure/csx?page=', page_number=1):
    """
    Generate a request on senacs page for a given 
    page number to get the name and location of 
    each social center.
    """

    url_page = url + str(page_number)
    request_text = request.urlopen(url_page).read()
    page = BeautifulSoup(request_text, 'html.parser')

    table = page.find('table', {'class': 'table table-striped table-hover'}).findAll('td')

    # fill the values of id, name and city
    ids = [id.getText() for id in table[::4]]
    names = [name.getText() for name in table[1::4]]
    cities = [city.getText() for city in table[2::4]]

    df = pd.DataFrame({'id': ids, 
                      'nom': names,
                      'ville': cities}
                      )
    df['page'] = page_number

    return df


def stack_pages(pages=range(91)):
    """
    Concatenate all empruntis scrapped data per region.
    """

    data = pd.DataFrame()    # empty Dataframe to fill
    for page_number in pages:
        temp = scrap_table_page(page_number=page_number)
        data = pd.concat([data, temp], axis=0)

    return data

In [ ]:
data = stack_pages()

'scrap_table_page'  1572.90 ms
'scrap_table_page'  1535.71 ms
'scrap_table_page'  1524.97 ms
'scrap_table_page'  1316.28 ms
'scrap_table_page'  1637.86 ms
'scrap_table_page'  1533.04 ms
'scrap_table_page'  1658.94 ms
'scrap_table_page'  1418.34 ms
'scrap_table_page'  1532.59 ms
'scrap_table_page'  1648.83 ms
'scrap_table_page'  1521.68 ms
'scrap_table_page'  1509.88 ms
'scrap_table_page'  1520.30 ms
'scrap_table_page'  1522.77 ms
'scrap_table_page'  1635.41 ms
'scrap_table_page'  1741.23 ms
'scrap_table_page'  1526.60 ms
'scrap_table_page'  1534.42 ms
'scrap_table_page'  1582.13 ms
'scrap_table_page'  1580.35 ms
'scrap_table_page'  1549.89 ms
'scrap_table_page'  1549.43 ms
'scrap_table_page'  2056.45 ms
'scrap_table_page'  1560.31 ms
'scrap_table_page'  1546.62 ms
'scrap_table_page'  1431.70 ms
'scrap_table_page'  1444.02 ms
'scrap_table_page'  2984.81 ms
'scrap_table_page'  1552.47 ms
'scrap_table_page'  1640.76 ms
'scrap_table_page'  1431.88 ms
'scrap_table_page'  1445.58 ms
'scrap_t

In [ ]:
data['code_postal'] = data['ville'].apply(lambda x: x.split('-')[0])
data['ville'] = data['ville'].apply(lambda x: x.split('-')[1])

In [ ]:
data

,id,nom,ville,page,code_postal
0,91,CENTRE SOCIAL DES GRANDES BORNES,Goussainville,0,95190
1,92,CENTRE SOCIAL ARCHIPELIA,PARIS 20EME ARRONDISSEMENT,0,75020
2,93,Espace Socioculturel Val de Charente,Ruffec,0,16700
3,94,CENTRE SOCIAL Maison Des Habitants Champaret,Bourgoin,0,38300
4,95,CENTRE SOCIAL MAISON DE LA CHALLE,Éragny,0,95610
...,...,...,...,...,...
6,16264,CENTRE SOCIOCULTUREL MUNICIPAL DE SAINT-DIZIER,Saint,90,52100
7,16267,ESPACE DE VIE SOCIALE L.E.P.H.A.R.E,Aniche,90,59580
8,16351,ASSOCIATION FERME BEAUREPAIRE,Boulogne,90,62200
9,16355,OFFICE DE LA JEUNESSE,Bruay,90,62700
